# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from citipy import citipy
#jupyter nbextension enable --py gmaps

# Import API key
from api_keys import g_key
# Import API key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# OpenWeatherMap API Key
api_key = g_key

In [4]:
api_key = weather_api_key

In [5]:
# Store filepath in a variable
# WeatherPyCities = "WeatherPy_data.csv"
#  # Read our Data file with the pandas library, including encoding
# WeatherPyCities = pd.read_csv("../Output_data/WeatherPy_data.csv, encoding="ISO-8859-1")
# WeatherPyCities.head()
WeatherPyCities = pd.read_csv("../WeatherPy/output_data/WeatherPy_data.csv", encoding="ISO-8859-1")
WeatherPyCities.head()       

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
0,0,Bengkulu,63,ID,1612535046,83,-3.8004,102.2655,78.80,4.14
1,1,Ushuaia,40,AR,1612534885,76,-54.8000,-68.3000,50.00,31.07
2,2,Rikitea,78,PF,1612535183,74,-23.1203,-134.9692,77.77,14.12
3,3,Atuona,25,PF,1612535184,80,-9.8000,-139.0333,78.51,14.43
4,4,Saskylakh,100,RU,1612535184,83,71.9167,114.0833,-31.61,5.57


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# create variables
locations = WeatherPyCities[["Lat", "Longitude"]]
weight = WeatherPyCities["Humidity"]

In [7]:
# create custom Heatmap - customized size
figure_layout = {
    'width': '900px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout)

In [9]:
# Plot Heatmap
fig = gmaps.figure(center=(40.0, -5.0), zoom_level=2)
max_intensity = np.max(weight)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
holiday_cities = WeatherPyCities[(WeatherPyCities["Cloudiness"]<30) & (WeatherPyCities["Humidity"]<30) &
 (WeatherPyCities["Max Temp"]>70) & (WeatherPyCities["Max Temp"]<85) & (WeatherPyCities["Wind Speed"]<10)]
holiday_cities

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
120,120,Awjilah,0,LY,1612535203,28,29.1081,21.2869,71.80,6.20
146,146,Jalu,0,LY,1612535206,28,29.0331,21.5482,72.05,6.20
192,192,San Rafael,0,AR,1612534997,25,-34.6177,-68.3301,79.00,6.42
398,398,Diego de Almagro,0,CL,1612535244,22,-26.3667,-70.0500,76.37,6.06


In [11]:
holiday_cities.reset_index(drop=True, inplace=True)
holiday_cities

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
0,120,Awjilah,0,LY,1612535203,28,29.1081,21.2869,71.80,6.20
1,146,Jalu,0,LY,1612535206,28,29.0331,21.5482,72.05,6.20
2,192,San Rafael,0,AR,1612534997,25,-34.6177,-68.3301,79.00,6.42
3,398,Diego de Almagro,0,CL,1612535244,22,-26.3667,-70.0500,76.37,6.06


In [12]:
holiday_cities = holiday_cities.drop('Unnamed: 0', axis=1)
holiday_cities

,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
0,Awjilah,0,LY,1612535203,28,29.1081,21.2869,71.80,6.20
1,Jalu,0,LY,1612535206,28,29.0331,21.5482,72.05,6.20
2,San Rafael,0,AR,1612534997,25,-34.6177,-68.3301,79.00,6.42
3,Diego de Almagro,0,CL,1612535244,22,-26.3667,-70.0500,76.37,6.06


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# add new column for hotel name
hotel_df = holiday_cities
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed,Hotel Name
0,Awjilah,0,LY,1612535203,28,29.1081,21.2869,71.80,6.20,
1,Jalu,0,LY,1612535206,28,29.0331,21.5482,72.05,6.20,
2,San Rafael,0,AR,1612534997,25,-34.6177,-68.3301,79.00,6.42,
3,Diego de Almagro,0,CL,1612535244,22,-26.3667,-70.0500,76.37,6.06,


In [14]:
# begin looping through hotel dataframe
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {"radius": 5000,
                  "types": "lodging",
                   "keyword": "Hotel",
                   "key": g_key}

        lat = row['Lat']
        long = row['Longitude']
        city = row['City']
        params['location'] = f"{lat}, {long}"

        hotel_name = requests.get(base_url, params=params).json()
        results = hotel_name['results']
        
        print(f"First hotel in {city} is {results[0]['name']}.") 
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except IndexError:
        print("No hotel found.")

No hotel found.
First hotel in Jalu is Jalu Hotel.
First hotel in San Rafael is Suter Petit Hotel.
First hotel in Diego de Almagro is Hotel Hambyll.


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Longitude"]]

In [19]:
# create custom Heatmap
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

# fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…